In [1]:
import sys,os,os.path
sys.path.append(os.path.expanduser('~/Codes/SolutionDistillation'))
import numpy as np
import tensorflow as tf
import os
from distill.common.hparams import TransformerHparam, LSTMHparam
from distill.data_util.prep_ptb import PTB

import tensorflow as tf
import numpy as np

from distill.common.hparams import TransformerHparam, LSTMHparam
from distill.data_util.prep_ptb import PTB
from distill.data_util.prep_sentwiki import SentWiki
from distill.models.lm_lstm import LmLSTM
import os
from distill.pipelines.lm import LMTrainer

# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
tf.app.flags.DEFINE_string("exp_name", "trial", "")
tf.app.flags.DEFINE_string("task_name", "sent_wiki", "ptb_lm")
tf.app.flags.DEFINE_string("log_dir", "logs", "")
tf.app.flags.DEFINE_string("save_dir", None, "")

tf.app.flags.DEFINE_string("model", "lm_lstm", "")
tf.app.flags.DEFINE_string("encoder_attention_dir", "top_down", "top_down | bottom_up")
tf.app.flags.DEFINE_integer("hidden_dim", 728, "")
tf.app.flags.DEFINE_integer("embedding_dim",728, "embeddings dim")
tf.app.flags.DEFINE_integer("output_dim", 8000, "")
tf.app.flags.DEFINE_integer("input_dim", 8000, "")
tf.app.flags.DEFINE_integer("number_of_heads", 4, "")
tf.app.flags.DEFINE_integer("ff_filter_size", 512, "")
tf.app.flags.DEFINE_float("initializer_gain", 1.0, "")
tf.app.flags.DEFINE_float("label_smoothing", 0.1, "")
tf.app.flags.DEFINE_boolean('train_embeddings', True, " False | True")
tf.app.flags.DEFINE_string('sent_rep_mode', "final", "none | final | all")
tf.app.flags.DEFINE_string('attention_mechanism',None, 'attention_mechanism')
tf.app.flags.DEFINE_integer("depth", 3, "")


tf.app.flags.DEFINE_float("input_dropout_keep_prob", 1.0, "")
tf.app.flags.DEFINE_float("hidden_dropout_keep_prob", 1.0, "")

tf.app.flags.DEFINE_float("learning_rate", 0.0001, "")
tf.app.flags.DEFINE_boolean("decay_learning_rate", True, "True | False")
tf.app.flags.DEFINE_float("l2_rate", 0.0001, "")

tf.app.flags.DEFINE_integer("batch_size", 64, "")
tf.app.flags.DEFINE_integer("training_iterations", 300000, "")

tf.app.flags.DEFINE_integer("vocab_size", 8000, "")
tf.app.flags.DEFINE_boolean("bidirectional", False, "If the LSTM layer is bidirectional")


tf.app.flags.DEFINE_string("pretrained_embedding_path", "data/sst/filtered_glove.txt", "pretrained embedding path")
tf.app.flags.DEFINE_string("data_path", "./data", "data path")

tf.app.flags.DEFINE_string("f","","kernel")
hparams = tf.app.flags.FLAGS

In [3]:
Models = {"lm_lstm": LmLSTM}
tasks = {'ptb_lm': PTB('../data/ptb'),
       'sent_wiki': SentWiki('../data/sent_wiki')}

In [4]:

task = tasks[hparams.task_name]
hparams.vocab_size = tasks[hparams.task_name].vocab_length
hparams.output_dim = len(tasks[hparams.task_name].target_vocab)
lstm_params = LSTMHparam(input_dim=hparams.input_dim,
                         hidden_dim=hparams.hidden_dim,
                         output_dim=hparams.output_dim,
                         encoder_depth=hparams.depth,
                         decoder_depth=0,
                         number_of_heads=hparams.number_of_heads,
                         ff_filter_size=hparams.ff_filter_size,
                       initializer_gain=hparams.initializer_gain,
                       batch_size=hparams.batch_size,
                       input_dropout_keep_prob=hparams.input_dropout_keep_prob,
                       hidden_dropout_keep_prob=hparams.hidden_dropout_keep_prob,
                       vocab_size=hparams.vocab_size,
                       label_smoothing=hparams.label_smoothing,
                       attention_mechanism=None,
                       sent_rep_mode=hparams.sent_rep_mode,
                       embedding_dim=hparams.embedding_dim,
                       train_embeddings = hparams.train_embeddings,
                       learning_rate=hparams.learning_rate)

model_params = {"lm_lstm": lstm_params}
model = Models[hparams.model](model_params[hparams.model],
                                  task=tasks[hparams.task_name],
                                  scope=hparams.model)

In [5]:
save_dir = '../logs/sent_wiki/'+ \
'lm_lstm_depth3_hidden_dim728_batch_size128_d3-5_run_38_ntied'
checkpoint = tf.train.latest_checkpoint(save_dir)
print(checkpoint)

../logs/sent_wiki/lm_lstm_depth3_hidden_dim728_batch_size128_d3-5_run_38_ntied/model.ckpt-300000


In [6]:
with tfe.restore_variables_on_create(checkpoint):
    model.create_vars(reuse=False)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [7]:
from tensorflow.contrib.framework.python.framework import checkpoint_utils
var_list = checkpoint_utils.list_variables(checkpoint)
for v in var_list:
    print(v)

('beta1_power', [])
('beta2_power', [])
('global_step', [])
('lm_lstm/InputEmbed/embedding_and_softmax/weights', [9920, 728])
('lm_lstm/InputEmbed/embedding_and_softmax/weights/AdamW', [9920, 728])
('lm_lstm/InputEmbed/embedding_and_softmax/weights/AdamW_1', [9920, 728])
('lm_lstm/InputEmbed/embedding_and_softmax/weights/ExponentialMovingAverage', [9920, 728])
('lm_lstm/OutputEmbed/embedding_and_softmax/weights', [9920, 728])
('lm_lstm/OutputEmbed/embedding_and_softmax/weights/AdamW', [9920, 728])
('lm_lstm/OutputEmbed/embedding_and_softmax/weights/AdamW_1', [9920, 728])
('lm_lstm/OutputEmbed/embedding_and_softmax/weights/ExponentialMovingAverage', [9920, 728])
('lm_lstm/lm_lstm/LSTM_Cells/rnn/multi_rnn_cell/cell_0/L0/bias', [2912])
('lm_lstm/lm_lstm/LSTM_Cells/rnn/multi_rnn_cell/cell_0/L0/bias/AdamW', [2912])
('lm_lstm/lm_lstm/LSTM_Cells/rnn/multi_rnn_cell/cell_0/L0/bias/AdamW_1', [2912])
('lm_lstm/lm_lstm/LSTM_Cells/rnn/multi_rnn_cell/cell_0/L0/bias/ExponentialMovingAverage', [2912])

In [76]:
sentences = []
#with tfe.restore_variables_on_create(checkpoint):
for i in np.arange(110):
    with tfe.restore_variables_on_create(checkpoint):
        with tf.variable_scope(model.scope, reuse=tf.AUTO_REUSE):
            number_of_sample = 1000
            initial_constant = task.word2id['<eos>']
            
            inputs = tf.ones((number_of_sample), dtype=tf.int32) * initial_constant
            inputs_length = tf.map_fn(lambda  x: 1, inputs)
            #samples = model.sample(sampling_initial_inputs, tf.map_fn(lambda  x: 1, sampling_initial_inputs))

            def compute_decoding_step_input(current_input):
                return None

        def projection_layer(hidden_states):
            return model.output_embedding_layer.linear(hidden_states) + model.output_bias
    
    lstm_decoder_output_dic = model.lstm.predict(inputs_length=inputs_length,
                                                          target_length=100,
                                                          compute_decoding_step_input_fn=compute_decoding_step_input,
                                                          input_embedding_layer=model.input_embedding_layer,
                                                          output_embedding_layer=projection_layer, eos_id=model.eos_id,
                                                          is_train=False,
                                                          initial_inputs=inputs)

    predictions  = lstm_decoder_output_dic['samples'][:,:,0]
    length = lstm_decoder_output_dic['outputs_lengths']
    lengths = []
    ind = 0
    i = 0
    for l,s in zip(list(length.numpy()),list(predictions.numpy())):
        #print(s)
        lengths.append(l)
        for w in s:
            if w not in task.id2word:
                print(w)
        sentences.append(' '.join(task.decode(s[:l], task.id2word)))
        if l < length.numpy()[ind]:
            ind = i
        i += 1

In [77]:
len(sentences)

110000

In [78]:
with open('train.txt', "w") as f:
    f.writelines('\n'.join(sentences))

In [26]:
with tfe.restore_variables_on_create(checkpoint):
    batch_size = 10
    beam_width = 5
    decoder_cell = model.lstm.multi_lstm_cell#tf.nn.rnn_cell.BasicLSTMCell(num_units)
    encoder_outputs = decoder_cell.zero_state(batch_size, dtype=tf.float32)
    tiled_encoder_outputs = tf.contrib.seq2seq.tile_batch(encoder_outputs, multiplier=beam_width)

    my_decoder = tf.contrib.seq2seq.BeamSearchDecoder(cell=decoder_cell,
                                                      embedding=model.input_embedding_layer.shared_weights,
                                                      start_tokens=tf.fill([batch_size],task.word2id['the']),
                                                      end_token=task.word2id['<eos>'],
                                                      initial_state=tiled_encoder_outputs,
                                                      beam_width=beam_width)

     # dynamic decoding
    outputs, final_context_state, _ = tf.contrib.seq2seq.dynamic_decode(my_decoder,
                                                                       maximum_iterations=10,
                                                                       output_time_major=True)
    final_predicted_ids = outputs.predicted_ids
    scores = outputs.beam_search_decoder_output.scores.numpy().transpose()
    predicted_ids = outputs.beam_search_decoder_output.predicted_ids
    parent_ids = outputs.beam_search_decoder_output.parent_ids


In [27]:
predicted_ids = outputs.beam_search_decoder_output.predicted_ids
for s in predicted_ids.numpy().transpose()[np.argmax(np.sum(scores, axis=-1))]:
    print(task.decode(s, task.id2word))

['far', 'points', 'points', 'car', 'car', 'car', 'parts', 'parts', 'parts', 'parts']
['far', 'points', 'points', 'car', 'car', 'car', 'parts', 'parts', 'parts', 'parts']
['far', 'points', 'points', 'car', 'car', 'car', 'parts', 'parts', 'parts', 'parts']
['far', 'points', 'points', 'car', 'car', 'car', 'parts', 'parts', 'parts', 'parts']
['far', 'points', 'points', 'car', 'car', 'car', 'parts', 'parts', 'parts', 'parts']
['far', 'points', 'points', 'car', 'car', 'car', 'parts', 'parts', 'parts', 'parts']
['far', 'points', 'points', 'car', 'car', 'car', 'parts', 'parts', 'parts', 'parts']
['far', 'points', 'points', 'car', 'car', 'car', 'parts', 'parts', 'parts', 'parts']
['far', 'points', 'points', 'car', 'car', 'car', 'parts', 'parts', 'parts', 'parts']
['far', 'points', 'points', 'car', 'car', 'car', 'parts', 'parts', 'parts', 'parts']


In [ ]:
with tfe.restore_variables_on_create(checkpoint):
    embeddings = model.input_embedding_layer.shared_weights

In [ ]:
embeddings[0]

In [ ]:
model.embedding_layer.shared_weights

In [ ]:
dataset = tf.data.TFRecordDataset(task.get_tfrecord_path(mode="train"))
dataset = dataset.map(task.parse_examples)
dataset = dataset.padded_batch(hparams.batch_size, padded_shapes=task.get_padded_shapes())
dataset = dataset.shuffle(buffer_size=1000)
dataset = dataset.repeat()
train_iterator = dataset.make_one_shot_iterator()

In [ ]:
examples = train_iterator.next()

In [ ]:
for e in examples[0].numpy():
    print(task.decode(e, task.id2word))